In [2]:

import numpy as np
from  keras.utils import np_utils
from matplotlib import pyplot as plt
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [3]:
def federated_train_images(node, data, label, size, file='renameTHIS.h'):
    # test
    with open(file, 'w') as f:
        num_of_image = size
        for i in range(num_of_image):
            f.write('#define NODE_%d_TRAIN_IMG_%d {'%(node,i) )
            np.round(data[i]).flatten().tofile(f, sep=", ", format="%d") # convert 0~1 to 0~127
            f.write('} \n')
            f.write('#define NODE_%d_TRAIN_IMG%d_LABEL'% (node, i))
            f.write(' %d \n \n' % label[i])
        f.write('#define NODE_%d_TOTAL_TRAIN_IMAGES %d \n \n'%(node,num_of_image))

        f.write('static q7_t NODE_%d_TRAIN_IMAGES[%d][%d] = {' % (node,num_of_image, data[0].flatten().shape[0]))
        f.write('NODE_%d_TRAIN_IMG_0'%(node))
        for i in range(num_of_image -1):
            f.write(',NODE_%d_TRAIN_IMG_%d'%(node, i+1))
        f.write('};\n\n')

        f.write('static q7_t NODE_%d_TRAIN_LABELS[%d] = {' % (node,num_of_image))
        f.write('NODE_%d_TRAIN_IMG0_LABEL'%(node))
        for i in range(num_of_image -1):
            f.write(',NODE_%d_TRAIN_IMG%d_LABEL'%(node, i+1))
        f.write('};\n\n')

In [4]:
def imagesTestcfile(data, label, size, file='renameTHIS.h'):
    # test
    with open(file, 'w') as f:
        num_of_image = size
        print(num_of_image)
        for i in range(num_of_image):
#             selected = np.random.randint(0, 10000) # select 10 out of 1000.
            f.write('#define TEST_IMG%d {'% (i))
            np.round(data[i]).flatten().tofile(f, sep=", ", format="%d") # convert 0~1 to 0~127
            f.write('} \n')
            f.write('#define TEST_IMG%d_LABEL'% (i))
            f.write(' %d \n \n' % label[i])
        f.write('#define TOTAL_TEST_IMAGES %d \n \n'%(num_of_image))

        f.write('static q7_t TEST_IMAGES[%d][%d] = {' % (num_of_image, data[0].flatten().shape[0]))
        f.write('TEST_IMG0')
        for i in range(num_of_image -1):
            f.write(',TEST_IMG%d'%(i+1))
        f.write('};\n\n')

        f.write('static q7_t TEST_LABELS[%d] = {' % (num_of_image))
        f.write('TEST_IMG0_LABEL')
        for i in range(num_of_image -1):
            f.write(',TEST_IMG%d_LABEL'%(i+1))
        f.write('};\n\n')

## HAR Samples

In [5]:
columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
df = pd.read_csv('WISDM_ar_v1.1_raw.txt', header = None, names = columns, comment=';')
df = df.dropna()
# df.head()
# df.info()
N_TIME_STEPS = 90
N_FEATURES = 3
step = 20
segments = []
labels = []
for i in range(0, len(df) - N_TIME_STEPS, step):
    xs = df['x-axis'].values[i: i + N_TIME_STEPS]
    ys = df['y-axis'].values[i: i + N_TIME_STEPS]
    zs = df['z-axis'].values[i: i + N_TIME_STEPS]
    label = stats.mode(df['activity'][i: i + N_TIME_STEPS])[0][0]
    segments.append([xs, ys, zs])
    
    labels.append(label)


reshaped_segments = np.asarray(segments,dtype = np.float32).reshape(-1, N_TIME_STEPS, N_FEATURES)
# print(reshaped_segments.shape)
# print(labels)
labels = LabelEncoder().fit_transform(labels)

FileNotFoundError: [Errno 2] File WISDM_ar_v1.1_raw.txt does not exist: 'WISDM_ar_v1.1_raw.txt'

In [11]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# subset_x_train = x_train[np.isin(y_train, [0,1,2,3,4,5,6]).flatten()]
# subset_y_train = y_train[np.isin(y_train, [0,1,2,3,4,5,6]).flatten()]

# subset_x_test = x_test[np.isin(y_test, [0,1,2,3,4,5,6]).flatten()]
# subset_y_test = y_test[np.isin(y_test, [0,1,2,3,4,5,6]).flatten()]

# subset_x_train.shape[0]
# federated_train_images(0, subset_x_train*127, subset_y_train,subset_x_train.shape[0] , file='MNIST-TrainSet.h')
# imagesTestcfile(subset_x_test*127, subset_y_test, subset_x_test.shape[0], file='MNIST-TestSet.h')

# x_train.shape[0]
federated_train_images(0,x_train[:2000]*127, y_train[:2000], 2000 , file='MNIST-TrainSet.h')

# x_test.shape[0]
imagesTestcfile(x_test[:400]*127, y_test[:400], 400, file='MNIST-TestSet.h')




print ("done")

400
done


In [10]:
print(subset_x_train.shape[0])

NameError: name 'subset_x_train' is not defined